# Graph Convolutional Network by Kipf and Welling

## Imports

In [1]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

import pickle
import numpy as np

import itertools

import Notebooks.performance as pf

## GNN Definition

In [2]:
class LinearModule(nn.Module):
    """The linear transformation part of the GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(LinearModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation # This is the activation function

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

In [3]:
class GCN(nn.Module):
    """A GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = LinearModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(message_func=fn.copy_src(src='h', out='m'), reduce_func=fn.sum(msg='m', out='h'))
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [4]:
class Net(nn.Module):
    def __init__(self, infeats, hidden_size, outfeats):
        super(Net, self).__init__()
        self.gcn1 = GCN(infeats, hidden_size, F.relu)
        self.gcn2 = GCN(hidden_size, outfeats, lambda x: F.log_softmax(x,1))
        self.dropout = nn.Dropout(0.2)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        #x = self.dropout(x)
        x = self.gcn2(g, x)
        return x

## Data Loading

In [5]:
from dgl.data import citation_graph as citegrh
import networkx as nx

data = citegrh.load_cora()
features = th.FloatTensor(data.features)
labels = th.LongTensor(data.labels)
mask = th.ByteTensor(data.train_mask)
g = data.graph

# add self loop
g.remove_edges_from(nx.selfloop_edges(g))
g = DGLGraph(g)
g.add_edges(g.nodes(), g.nodes())

## Select Training Set

In [6]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

## Training

In [8]:
import time

net = Net(features.shape[1], 21, len(np.unique(labels)))
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
net.train() # Set to training mode (use dropout)

dur = []
for epoch in range(150):
    if epoch >=3:
        t0 = time.time()

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = net(g, features)
    #scores = th.tensor([F.nll_loss(prediction.detach()[1-mask][:,p], labels[1-mask]) for p in label_perms])
    #val_loss = th.min(scores)
    #val_loss = F.nll_loss(prediction.detach()[1-mask], labels[1-mask])
    val_loss=th.tensor(np.nan)
    net.train()

    # Compute loss for train nodes
    logits = net(g, features)

    loss = th.tensor(1000.0,requires_grad=True)
    for p in label_perms:
        loss = th.min(loss,F.nll_loss(logits[mask][:,p], labels[mask]))

    #loss = F.nll_loss(logits[mask], labels[mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Val.Loss {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), val_loss.item(), np.mean(dur)))

C:\Users\a_liso02\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\a_liso02\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Epoch 00000 | Loss 1.8940 | Val.Loss nan | Time(s) nan
Epoch 00001 | Loss 1.8565 | Val.Loss nan | Time(s) nan
Epoch 00002 | Loss 1.8246 | Val.Loss nan | Time(s) nan
Epoch 00003 | Loss 1.7983 | Val.Loss nan | Time(s) 1.0251
Epoch 00004 | Loss 1.7706 | Val.Loss nan | Time(s) 1.0251
Epoch 00005 | Loss 1.7418 | Val.Loss nan | Time(s) 1.0294
Epoch 00006 | Loss 1.7126 | Val.Loss nan | Time(s) 1.0236
Epoch 00007 | Loss 1.6834 | Val.Loss nan | Time(s) 1.0273
Epoch 00008 | Loss 1.6549 | Val.Loss nan | Time(s) 1.0296
Epoch 00009 | Loss 1.6268 | Val.Loss nan | Time(s) 1.0278
Epoch 00010 | Loss 1.5995 | Val.Loss nan | Time(s) 1.0396
Epoch 00011 | Los

In [9]:
# Visualise predictions
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
a = np.transpose(np.vstack([final_prediction[mask].numpy().argmax(axis=1),labels[mask].numpy()]))
a[a[:,0].argsort()]
# as can be seen, the net predicts other labels, but gets the clusters right :)

array([[0, 6],
       [0, 6],
       [0, 6],
       [0, 6],
       [0, 6],
       [0, 6],
       [0, 6],
       [1, 4],
       [1, 4],
       [1, 4],
       [1, 4],
       [1, 4],
       [1, 4],
       [1, 4],
       [2, 5],
       [2, 5],
       [3, 1],
       [3, 1],
       [3, 1],
       [3, 1],
       [3, 1],
       [3, 1],
       [3, 1],
       [3, 1],
       [3, 1],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [4, 3],
       [5, 0],
       [5, 0],
       [5, 0],
       [5, 0],
       [5, 0],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2],
       [6, 2]], dtype=int64)

## Evaluation

In [13]:
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()
pf.performance_as_df(labels,final_prediction,mask)

,All,Train,Test
Mutual Information,0.335656,1.0,0.326277
Rand-Index,0.303419,1.0,0.294430
Variation of Information,2.279773,0.0,2.306509
